In [ ]:
import cv2
import os

label_name = input("Enter the label name:-")//27
num_samples = int(input("Enter the number of samples:- ")) //140

IMG_SAVE_PATH = 'Dataset'
IMG_CLASS_PATH = os.path.join(IMG_SAVE_PATH, label_name)

try:
    os.makedirs(IMG_SAVE_PATH, exist_ok=True)
    os.makedirs(IMG_CLASS_PATH, exist_ok=True)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))

cap = cv2.VideoCapture(0)

start = False
count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    if count == num_samples:
        break

    frame = cv2.flip(frame, 1)
    cv2.rectangle(frame, (300, 50), (600, 350), (255, 255, 255), 2)

    if start:
        roi = frame[50:350, 300:600] #region of intrest

        # Save the captured image without preprocessing
        save_path = os.path.join(IMG_CLASS_PATH, '{}{}.jpg'.format(label_name, count + 1))
        cv2.imwrite(save_path, roi)

        count += 1

    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, "Collecting {}".format(count), (5, 50), font, 0.7, (0, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow("Collecting images", frame)

    k = cv2.waitKey(10)
    if k == ord('a'):
        start = not start

    if k == ord('q'):
        break

print("\n{} image(s) saved to {}".format(count, IMG_CLASS_PATH))
cap.release()
cv2.destroyAllWindows()



140 image(s) saved to Dataset\None


In [1]:
from google.colab import drive
drive.mount('/content/drive') #google collab

Mounted at /content/drive


In [ ]:
#Preprocessing
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def preprocess_image(image):
    # Apply Gaussian blur to reduce noise
    blurred_image = cv2.GaussianBlur(image, (1, 1), 0)

    # Convert the image to RGB
    rgb_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2RGB)

    # Split the RGB channels
    r, g, b = cv2.split(rgb_image)

    # Apply adaptive thresholding to each channel
    _, thresholded_r = cv2.threshold(r, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, thresholded_g = cv2.threshold(g, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, thresholded_b = cv2.threshold(b, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Merge the thresholded channels back into an RGB image
    thresholded_image = cv2.merge([thresholded_r, thresholded_g, thresholded_b])

    # Resize the image to match model input size
    resized_image = cv2.resize(thresholded_image, (224, 224))

    # Normalize pixel values to range [0, 1]
    normalized_image = resized_image / 255.0

    # Add batch dimension
    reshaped_image = np.expand_dims(normalized_image, axis=0)

    return reshaped_image

# def preprocess_image(image):
#     # Apply Gaussian blur to reduce noise
#     blurred_image = cv2.GaussianBlur(image, (1, 1), 0)

#     # Convert the image to RGB
#     rgb_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2RGB)

#     # Resize the image to match model input size
#     resized_image = cv2.resize(rgb_image, (64, 64))

#     # Normalize pixel values to range [0, 1]
#     normalized_image = resized_image / 255.0

#     return normalized_image

def preprocess_images_in_directory(input_dir, output_dir):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    for label in os.listdir(input_dir):
        label_path = os.path.join(input_dir, label)
        if os.path.isdir(label_path):
            for filename in tqdm(os.listdir(label_path), desc=f"Processing {label}"):
                if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
                    image_path = os.path.join(label_path, filename)
                    image = cv2.imread(image_path)
                    if image is not None:
                        preprocessed_image = preprocess_image(image)
                        output_path = os.path.join(output_dir, label, filename)
                        os.makedirs(os.path.dirname(output_path), exist_ok=True)
                        cv2.imwrite(output_path, (preprocessed_image[0] * 255).astype(np.uint8))  # Convert to uint8 before saving
                    else:
                        print(f"Error loading image: {image_path}")

if __name__ == "__main__":
    input_dir = "/content/drive/MyDrive/Colab Notebooks/PROJ_HCI/Dataset"  # Specify the input directory containing images
    output_dir = "/content/drive/MyDrive/Colab Notebooks/PROJ_HCI/PreprocessedDataset"  # Specify the output directory for preprocessed images

    preprocess_images_in_directory(input_dir, output_dir)

    print("Preprocessing completed.")


Processing A: 100%|██████████| 140/140 [00:06<00:00, 20.41it/s]

Preprocessing completed.


In [4]:
#splitting
import os
import shutil
from sklearn.model_selection import train_test_split

def split_dataset(input_dir, output_dir, test_size=0.1, val_size=0.1, random_state=42):
    # Create output directories if they don't exist
    train_dir = os.path.join(output_dir, 'train')
    test_dir = os.path.join(output_dir, 'test')
    val_dir = os.path.join(output_dir, 'val')
    for directory in [train_dir, test_dir, val_dir]:
        os.makedirs(directory, exist_ok=True)

    for label in os.listdir(input_dir):
        label_path = os.path.join(input_dir, label)
        if os.path.isdir(label_path):
            # Split images for the current label
            images = os.listdir(label_path)
            X_train, X_test_val = train_test_split(images, test_size=test_size+val_size, random_state=random_state)
            X_val, X_test = train_test_split(X_test_val, test_size=val_size/(test_size+val_size), random_state=random_state)

            # Copy images to train, test, and val directories
            for filename in X_train:
                src = os.path.join(label_path, filename)
                dst = os.path.join(train_dir, label)
                os.makedirs(dst, exist_ok=True)
                shutil.copy(src, dst)
            for filename in X_test:
                src = os.path.join(label_path, filename)
                dst = os.path.join(test_dir, label)
                os.makedirs(dst, exist_ok=True)
                shutil.copy(src, dst)
            for filename in X_val:
                src = os.path.join(label_path, filename)
                dst = os.path.join(val_dir, label)
                os.makedirs(dst, exist_ok=True)
                shutil.copy(src, dst)

if __name__ == "__main__":
    input_dir = "/content/drive/MyDrive/Colab Notebooks/PROJ_HCI/PreprocessedDataset"  # Specify the input directory containing preprocessed images
    output_dir = "/content/drive/MyDrive/Colab Notebooks/PROJ_HCI/SplitDataset"  # Specify the output directory for split dataset

    split_dataset(input_dir, output_dir)

    print("Dataset split completed.")
import os

def count_images_in_directory(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.png'):
                count += 1
    return count

if __name__ == "__main__":
    output_dir = r"C:\Users\janas\Downloads\PROJ_HCI_Collab-20240412T130910Z-001\PROJ_HCI_Collab\SplitDataset"  # Specify the output directory for split dataset

    train_dir = os.path.join(output_dir, 'train')
    test_dir = os.path.join(output_dir, 'test')
    val_dir = os.path.join(output_dir, 'val')

    num_train_images = count_images_in_directory(train_dir)
    num_test_images = count_images_in_directory(test_dir)
    num_val_images = count_images_in_directory(val_dir)

    print("Number of images in train directory:", num_train_images)
    print("Number of images in test directory:", num_test_images)
    print("Number of images in val directory:", num_val_images)


Dataset split completed.
Number of images in train directory: 3024
Number of images in test directory: 378
Number of images in val directory: 378
